### Porting to Google Colab
The following cell enables this notebook to run from Google Colab as well as from your local machine IDE.<br>
You can change `root_directory` and/or `this_notebook_google_path` to point to the directory in your Google account, which contains this notebook, together with the `imgs` sub-directory and the rest of the files.<br>

In [1]:
import sys
import os
try:
    from google.colab import drive as google_drive # type: ignore
except:
    # no Google Colab --> fall back to local machine
    google_drive = None

if google_drive is not None:
    google_drive_directory = os.path.join('/','content','gdrive')
    google_drive.mount(google_drive_directory)
    all_projects_path = os.path.join(google_drive_directory, 'Othercomputers','My Laptop', 'projects')
else:
    all_projects_path = os.path.join('d:\\', 'projects')

project_path = os.path.join(all_projects_path,'RUNI','Thesis')
assert os.path.exists(project_path), f'Project path {project_path} not found!'
# enable import python files from this notebook's path
sys.path.append(project_path)
# enable reading images and data files from this notebook's path
os.chdir(project_path)

datasets_path = os.path.join(project_path, 'datasets')
assert os.path.exists(datasets_path), f'Datasets path {datasets_path} not found!'

output_path = os.path.join(project_path, 'output')
os.makedirs(output_path, exist_ok=True)
assert os.path.exists(output_path), f'Output path {output_path} not found!'

print(f'Current working directory: {os.getcwd()}')
print(f'Datasets path: {datasets_path}')
print(f'Output path: {output_path}')

Current working directory: d:\projects\RUNI\Thesis
Datasets path: d:\projects\RUNI\Thesis\datasets
Output path: d:\projects\RUNI\Thesis\output


In [2]:
from python.hpc import HybridArray

Numba version: 0.60.0
numba.njit is available.
CUDA is available and will be used for GPU operations.
Printing CUDA active device attributes:
    Name:                               NVIDIA GeForce GTX 1650
    Free Memory:                        3367680 [KB]
    Total Memory:                       4193984 [KB]
    Compute capability:                 7.5
    Clock rate:                         1560.00 [MHz]
    Memory clock rate:                  4001.00 [MHz]
    Memory bus width:                   128 bits
    Memory band width (theoretical)     128.03 [GByte/Sec]
    Number of multiprocessors:          16
    Minimal grid size:                  128
    Maximum grid size:                  (2147483647, 65535, 65535)
    Maximum block dimensions:           (1024, 1024, 64)
    Maximum threads per block:          1024
    Warp size:                          32
    Maximum shared memory per block:    49152 [bytes]
    Maximum registers per block:        65536
    Total constant memory:   

In [3]:
import numpy as np
from python.metrics.metrics import detect_signal_auc, create_noise_4_auc, create_signal_4_auc
from python.metrics.metrics import analyze_multi_auc

In [4]:
shape=(20,100)
n1s = (np.arange(0.01,0.11,0.01)*shape[1]).astype(np.uint32)
mus = np.arange(0.1,1.1,0.1,dtype=np.float64)


In [5]:

def simulation(shape: tuple, gpu: bool, njit: bool,\
               transform_method: str,\
               discovery_method: str,\
               n1: np.uint32,\
               mu: float = 1.0) -> tuple[np.ndarray, np.ndarray]:
    with (
        HybridArray() as signal,
        HybridArray() as noise,
        HybridArray() as auc_row
    ):
        create_noise_4_auc(noise=noise, shape=shape,\
                           transform_method=transform_method,\
                           discover_method=discovery_method,\
                           use_gpu=gpu, use_njit=njit)
        create_signal_4_auc(signal=signal, shape=shape,\
                            transform_method=transform_method,\
                            discover_method=discovery_method,\
                            ind_model=1,\
                            n1=n1, mu=mu, use_gpu=gpu, use_njit=njit)
        auc_row.realloc(like=signal, shape=(1,shape[1]))
        detect_signal_auc(\
            noise_input=noise,\
            signal_input_work=signal,\
            auc_out_row=auc_row,\
            use_njit=njit)
        ret = (signal.numpy(), auc_row.numpy())
    return ret

def simulation3(shape: tuple, transform_method: str, discovery_method: str,\
                n1: np.uint32, mu: float = 1.0) -> None:
    print(f'Running on {transform_method=} {discovery_method=} {shape=} {n1=} {mu=}')
    for gpu,njit in [(False,False), (False,True), (True,False)]:
        signal, auc =\
            simulation(shape=shape, gpu=gpu, njit=njit,\
                       transform_method=transform_method,\
                       discovery_method=discovery_method,\
                       n1=n1, mu=mu)
        print(f'{gpu=} {njit=} --> signal.mean={signal.mean():.2e} auc.mean()={auc.mean():.2f}')




In [6]:
simulation3(shape=shape, transform_method='higher_criticism_stable',\
            discovery_method='argmin', n1=n1s[0], mu=mus[0])

Running on transform_method='higher_criticism_stable' discovery_method='argmin' shape=(20, 100) n1=1 mu=0.1
gpu=False njit=False --> signal.mean=-1.23e+00 auc.mean()=0.67
gpu=False njit=True --> signal.mean=-1.23e+00 auc.mean()=0.67


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


gpu=True njit=False --> signal.mean=3.35e-01 auc.mean()=0.67


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [7]:
simulation3(shape=shape, transform_method='higher_criticism_unstable',\
            discovery_method='argmin', n1=n1s[0], mu=mus[0])

Running on transform_method='higher_criticism_unstable' discovery_method='argmin' shape=(20, 100) n1=1 mu=0.1
gpu=False njit=False --> signal.mean=-2.01e+00 auc.mean()=0.58
gpu=False njit=True --> signal.mean=-2.01e+00 auc.mean()=0.58
gpu=True njit=False --> signal.mean=4.23e-01 auc.mean()=0.58


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [8]:
simulation3(shape=shape, transform_method='berk_jones',\
            discovery_method='argmin', n1=n1s[0], mu=mus[0])

Running on transform_method='berk_jones' discovery_method='argmin' shape=(20, 100) n1=1 mu=0.1
gpu=False njit=False --> signal.mean=1.49e-01 auc.mean()=0.63
gpu=False njit=True --> signal.mean=1.49e-01 auc.mean()=0.63


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 14 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


gpu=True njit=False --> signal.mean=3.66e-01 auc.mean()=0.63


In [9]:

def simulation_multi(shape: tuple, gpu: bool, njit: bool,\
               transform_method: str,\
               discovery_method: str,\
               n1s: list|np.ndarray,\
               mus: list|np.ndarray) -> np.ndarray:
    with HybridArray() as auc_result:
        analyze_multi_auc(auc_results=auc_result, shape=shape,\
                          transform_method=transform_method,\
                          discover_method=discovery_method,\
                            n1s=n1s, mus=mus,\
                                use_gpu=gpu, use_njit=njit)
        ret = auc_result.numpy()
    return ret

def simulation3_multi(shape: tuple, transform_method: str,\
                    discovery_method: str,\
                        n1s: list|np.ndarray, mus: list|np.ndarray) -> None:
    print(f'Running on {transform_method=} {discovery_method=} {shape=} {n1s=} {mus=}')
    for gpu,njit in [(False,False), (False,True), (True,False)]:
        auc =\
            simulation_multi(shape=shape, gpu=gpu, njit=njit,\
                       transform_method=transform_method,\
                       discovery_method=discovery_method,\
                       n1s=n1s, mus=mus)
        print(f'{gpu=} {njit=} --> auc.mean()={auc.mean():.2f}')




In [10]:
simulation3_multi(shape=shape, transform_method='higher_criticism_stable',\
            discovery_method='argmin',\
            n1s=n1s, mus=mus)

Running on transform_method='higher_criticism_stable' discovery_method='argmin' shape=(20, 100) n1s=array([ 1,  2,  3,  4,  5,  6,  6,  8,  9, 10], dtype=uint32) mus=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])


Processing: 100%|██████████| 11/11 [00:00<00:00, 11.58step/s, Current Step=10]


gpu=False njit=False --> auc.mean()=0.70


Processing: 100%|██████████| 11/11 [00:00<00:00, 312.39step/s, Current Step=10]


gpu=False njit=True --> auc.mean()=0.70


Processing:  18%|█▊        | 2/11 [00:00<00:00, 125.63step/s, Current Step=2]d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
Processing:  73%|███████▎  | 8/11 [00:00<00:00, 163.48step/s, Current Step=8]d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
Processing: 100%|██████████| 11/11 [00:00<00:00, 170.37step/s, Current Step=10]

gpu=True njit=False --> auc.mean()=0.70
